In [ ]:
"""
Datto RMM - Update or Create Site and Account Variables from External Data

Author: Gabe McWilliams
Version: 1.0

Description:
Automates the creation, updating, or deletion of Datto RMM Site Variables and Account Variables based on:
- External client renaming standards
- Sophos Central tenant information
- Sophos installer arguments extraction

Process Overview:
- Pulls all active sites and variables from Datto RMM via API.
- Matches and cleans site/client names based on standard naming rules.
- Collects Sophos management server and token data for tenants.
- Constructs silent install arguments for both Windows and macOS endpoints.
- Injects new variables into Datto RMM, updates existing ones, and optionally removes old entries.

Technologies:
- Python 3
- Pandas
- Datto RMM REST API
- Sophos Central API
- Selenium (minor use if necessary)

Notes:
- Requires env.ini secret config for Datto RMM and Sophos Central API credentials.
- Assumes standardized Sophos CSV exports available in specified folders.
"""


<h1> DattoRMM - Update / Create Site Variables based on Dict </h1>

# Import Modules and Env

In [ ]:
# data import and file manipulation
import os
import json
import csv


#data conditioning
import pandas as pd
import numpy as np
import re
import datetime as dt

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# git repo folder
git_folder = 'd:/git/example_infrastructure_data_dev'

# dictionary Directory
dictionary_dir = 'd:/git/example_infrastructure_data_dev/dictionaries'

# source dir for nable exported data
source_folder = 'd:/project_docs/abc_nable_migration/abc_nable_exports/patch_management'

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/'

In [ ]:
# import configparser for env secrets
from configparser import ConfigParser

config = ConfigParser()
config.read(f'{git_folder}/config/env.ini')
import requests
from requests.structures import CaseInsensitiveDict

## Client Renaming Functions

In [ ]:
df = pd.read_parquet(f'D:/Git/data_parsing/dictionaries/standard_client_names.parquet')
client_rename_list = []

for index, row in df.iterrows():
    client_info_dict = {}
    client_info_dict['[REDACTED]'] = row['[REDACTED]']
    client_info_dict['currentName'] = row['currentName']

    client_rename_list.append(client_info_dict)

In [ ]:
cu_dict = {'Federal Credit Union':'FCU','Credit Union':'CU'}
def reword_creditunion(string):
    reword = string
    for k, v in cu_dict.items():
        if k in string:
            reword = re.sub(k,v,string)
            break
    return reword

In [ ]:
def client_names(c_name):
    sitename = c_name
    for client in client_rename_list:
        try:
            if client['[REDACTED]'] == c_name:
                # print(f"Previous: [{client['[REDACTED]']}] and Current: [{client['currentName']}]")
                sitename = client['currentName']
                break
        except Exception as e:
            break
    return sitename

In [ ]:
def active_site(site):
    if site['billingType'] == 'trial':
        return None
    elif site['dataRegion'] == None:
        return None
    elif re.match(r'inactive',str(site['showAs']).lower()):
        return None
    elif site['status'] == 'active':
        return site

# Create DattoRMM DataFrames

In [ ]:
# import and assign secrets from env.ini
dattormm_config = json.loads(os.environ.get("DATTO_RMM_CONFIG"))
base_uri = dattormm_config['base_uri']

## Create auth token

In [ ]:
# call token api url
token_uri = f'{base_uri}/auth/oauth/token'


# construct header
headers = CaseInsensitiveDict()
headers['Content-Type'] = 'application/x-www-form-urlencoded'

# construct req body
data = CaseInsensitiveDict()
data['grant_type'] = 'password'
data['username'] = dattormm_config['api_key']
data['password'] = dattormm_config['api_secret']

# request content response
resp = requests.post(token_uri, headers=headers, data=data, auth=('public-client', 'public'))
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

datto_access_token = c_dict['access_token']

## Pull Site Info

In [ ]:

## Create Devices DataFrame
# request content response
request_url = f'{base_uri}/api/v2/account/sites'

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {datto_access_token}'
headers['Content-Type'] = 'application/json'

# construct req body
data = ''

print(f'Request URL: {request_url}')

resp = requests.get(request_url, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

df_sites = pd.DataFrame(c_dict['sites'])

In [ ]:
df_site_info = df_sites[df_sites['name'] != 'Deleted Devices']

In [ ]:
dattormm_client_sites = []
for index, row in df_site_info.iterrows():
    temp_dict = {}
    temp_dict['siteName'] = row['name']
    temp_dict['siteUid'] = row['uid']
    dattormm_client_sites.append(temp_dict)

# Pull all ACCOUNT Variables as a DataFrame

In [ ]:
# request content response
request_url = f'{base_uri}/api/v2/account'

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {datto_access_token}'
headers['Content-Type'] = 'application/json'

# construct req body
data = ''

print(f'Request URL: {request_url}')

resp = requests.get(request_url, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

account_info = {}
account_info['accountName'] = c_dict['name']
account_info['accountUid'] = c_dict['uid']

In [ ]:
# request content response
request_url = f'{base_uri}/api/v2/account/variables'

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {datto_access_token}'
headers['Content-Type'] = 'application/json'

# construct req body
data = ''

print(f'Request URL: {request_url}')

resp = requests.get(request_url, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

df_account_variables = pd.DataFrame(c_dict['variables'])
df_account_variables['accountName'] = account_info['accountName']
df_account_variables['accountUid'] = account_info['accountUid']

# Pull all SITE Variables as a DataFrame

In [ ]:
df_site_variables = pd.DataFrame()

for site_info in dattormm_client_sites:
    print(site_info['siteUid'])
    # request content response
    request_url = f"{base_uri}/api/v2/site/{site_info['siteUid']}/variables"

    # construct header
    headers = CaseInsensitiveDict()
    headers['Authorization'] = f'Bearer {datto_access_token}'
    headers['Content-Type'] = 'application/json'

    # construct req body
    data = ''

    print(f'Request URL: {request_url}')

    resp = requests.get(request_url, headers=headers, data=data)
    content = resp.content.decode('utf-8')
    c_dict = json.loads(content)

    # iterate and combine remaining pages
    try:
        df_current_page = pd.DataFrame(c_dict['variables'])
        df_current_page['siteName'] = site_info['siteName']
        df_current_page['siteUid'] = site_info['siteUid']

        try:
            df_site_variables = pd.concat([df_site_variables, df_current_page], ignore_index=False)
        except:
            df_site_variables = df_current_page

    except:
        pass

In [ ]:
df_datto_variables = pd.concat([df_account_variables,df_site_variables], ignore_index=True)

In [ ]:
report_creation_date = (dt.datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))
df_datto_variables['reportCreationDate'] = report_creation_date

In [ ]:
df_datto_variables['variableId'] = df_datto_variables['id'].apply(lambda x: str(x).replace(".0",""))

# Initial Metrics and CSV Creation

## All Fields

In [ ]:
df_datto_variables.to_csv(export_folder + '.csv', index=False)

In [ ]:
df_datto_variables = df_datto_variables[['variableId','accountName', 'accountUid','siteName', 'siteUid', 'id', 'name', 'value', 'masked',
                                         'reportCreationDate']]

In [ ]:
df_datto_variables.to_csv(export_folder + 'datto_rmm_account_site_variables_' + str(current_time) + '.csv',index=False)

# Insert Variable or Update if Exists

## Import Current Site Variables Dict

In [ ]:
df = pd.read_csv(f"{dictionary_dir}/datto_rmm_standard_varables.dict")
df.fillna('[empty]',inplace=True)
site_var_std_list = []
account_var_std_list = []
for index, row in df.iterrows():
    row_dict = {}
    try:
        if row['accountName'] != '[empty]':
            row_dict['accountName'] = row['accountName']
            row_dict['accountUid'] = row['accountUid']
            row_dict['varId'] = row['id']
            row_dict['varName'] = row['name']
            row_dict['value'] = row['value']
            row_dict['masked'] = row['masked']
            account_var_std_list.append(row_dict)
        else:
            row_dict['siteName'] = row['siteName']
            row_dict['siteUid'] = row['siteUid']
            row_dict['varId'] = row['id']
            row_dict['varName'] = row['name']
            row_dict['value'] = row['value']
            row_dict['masked'] = row['masked']
            site_var_std_list.append(row_dict)
    except Exception as e:
        print(e)

# Create Sophos DataFrame from API for Active Account Filtering

In [ ]:
sophos_config = config['sophoscentral']

base_uri = sophos_config['base_uri']

In [ ]:
# call token api url
token_uri = 'https://id.sophos.com/api/v2/oauth2/token'

# construct header
headers = CaseInsensitiveDict()
headers["Content-Type"] = "application/x-www-form-urlencoded"

# construct req body
data = f"grant_type=client_credentials&client_id={sophos_config['client_id']}&client_secret={sophos_config['client_secret']}&scope=token"

# request content response
resp = requests.post(token_uri, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

# create auth and refresh tokens
sophos_access_token = c_dict['access_token']
sophos_refresh_token = c_dict['refresh_token']

## Partner ID Lookup

In [ ]:
# Partner ID lookup
whoami = 'https://api.central.sophos.com/whoami/v1'

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {sophos_access_token}'

# request partner id
resp = requests.get(whoami, headers=headers)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

# store partner id
partner_id = c_dict['id']

## Pull Tenant Info

### Filter Data for Active

In [ ]:
# Tenant ID lookup
tenants_url = 'https://api.central.sophos.com/partner/v1/tenants'

# construct header
headers = CaseInsensitiveDict()

headers['Authorization'] = f'Bearer {sophos_access_token}'
headers['X-Partner-ID'] = f'{partner_id}'
headers['Accept'] = 'application/json'
headers['pageSize'] = '100'


resp = requests.get(tenants_url,headers=headers)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)


sophos_tenants = []
all_sophos = []

for site in c_dict['items']:
    all_sophos.append(site)
    filtered_site = active_site(site)
    if filtered_site:
        name_cu_shaped = reword_creditunion(site['showAs'])
        name_standardized = client_names(name_cu_shaped)
        sophos_tenants.append(name_standardized)

In [ ]:
df = pd.DataFrame(all_sophos)
df

## Import Sophos CSV's

In [ ]:
sophos_csv_export_list = []

df_sophos_installer_info = pd.DataFrame()

for root, dirs, files in os.walk(f"{export_folder}/sophos_csv"):
    for file in files:
        df = pd.read_csv(f"{root}/{file}")
        df_sophos_installer_info = pd.concat([df, df_sophos_installer_info],ignore_index=True)

In [ ]:
df_sophos_installer_info

# Disassemble and Reassemble Sophos Info into Site Var Dict List

## Standardize Client Names

In [ ]:
def assign_siteid(sophosClient):
    for site in dattormm_client_sites:
        if site['siteName'] == sophosClient:
            return (site['siteUid'])
            break
        else:
            pass
    print(f"{sophosClient} not found")
    return '[Not in Datto]'

In [ ]:
def reconstruct_rows(row):
    row_dict = {}
    row_dict['originalName'] = row['Customer Name']

    client_name = client_names(row['Customer Name'])

    reworded_sitename = reword_creditunion(client_name)
    # print(reworded_sitename)
    row_dict['siteName'] = reworded_sitename.replace("'","")

    if re.match(r'sudo',row['Example Command Line']):
        row_dict['os'] = 'macOS'
        row_dict['varName'] = 'SophosMacOSArguments'
        row['silentVar'] = ''
    else:
        row_dict['os'] = 'winOS'
        row_dict['varName'] = 'SophosWindowsArguments'

    row_dict['installString'] = row['Example Command Line']

    row_dict['customerToken'] = row['Customer Token']

    row_dict['managementServer'] = row['Management Server']

    row_dict['products'] = (re.findall(r'--products[="\s]+([\w+\,\s]+)["\s]+\--', row['Example Command Line'])[0])

    row_dict['siteId'] = assign_siteid(row_dict['siteName'])

    return row_dict

In [ ]:
client_info_list = []

for index, row in df_sophos_installer_info.iterrows():
    if re.match(r'inactive',(row['Customer Name'].lower())):
        pass
    else:
        client_info_list.append(reconstruct_rows(row))

In [ ]:
df_sohos_install_info = pd.DataFrame(client_info_list)

In [ ]:
df_sohos_install_info.to_csv(f"{export_folder}.csv")

### Filter for only client rows with siteid

In [ ]:
df_sophos_data_injection = df_sohos_install_info[df_sohos_install_info['siteId'] != '[Not in Datto]']
df_sohos_install_info[df_sohos_install_info['siteId'] != '[Not in Datto]'].to_csv('.csv')

# Install Variables

## Argument Templates

In [ ]:
def install_arguments(var_info):

    if var_info['os'] == 'winOS':
        argument_template = f"""--customertoken="{var_info['customerToken']}" --epinstallerserver="{var_info['managementServer']}" --products="{var_info['products']}" --quiet"""
    else:
        argument_template = f"""--customertoken {var_info['customerToken']} --mgmtserver {var_info['managementServer']} --products {var_info['products']} --quiet"""

    return argument_template

In [ ]:
df_sophos_data_injection

In [ ]:
for index, var_info in df_sophos_data_injection[:1].iterrows():
    print(var_info['siteName'])
    print(var_info['siteId'])
    print(var_info['os'])
    print(install_arguments(var_info))
    print(var_info['varName'])


In [ ]:
# df_test = df_datto_variables[df_datto_variables['name'].str.contains('String')]
# df_test

In [ ]:
for index, row in df_test.iterrows():

    # request content response
    request_url = f"https://concord-api.centrastage.net/api/v2/site/{row['siteUid']}/variable/{row['variableId']}"



    # construct header
    headers = CaseInsensitiveDict()
    headers['Authorization'] = f'Bearer {datto_access_token}'
    headers['Content-Type'] = 'application/json'
    headers['accept'] = '*/*'


    # construct req body


    print(f'Request URL: {request_url}')



    resp = requests.delete(request_url, headers=headers)


    print(f"response code: {resp.status_code}")
    # print(resp.headers)
    print(resp.content)

In [ ]:
for index, var_info in df_sophos_data_injection.iterrows():
    # request content response
    request_url = f"https://concord-api.centrastage.net/api/v2/site/{var_info['siteId']}/variable"


    # construct header
    headers = CaseInsensitiveDict()
    headers['Authorization'] = f'Bearer {datto_access_token}'
    headers['Content-Type'] = 'application/json'
    headers['accept'] = '*/*'


    # construct req body
    json_data = {
        'name': var_info['varName'],
        'value': install_arguments(var_info),
        'masked': False,
    }

    print(f'Request URL: {request_url}')

    print(var_info['siteName'])

    resp = requests.put(request_url, headers=headers, json=json_data)

    print(json_data)

    print(f"response code: {resp.status_code}")
    # print(resp.headers)
    print(resp.content)

In [ ]:
for var_info in account_var_std_list:
    # request content response
    request_url = "https://concord-api.centrastage.net/api/v2/account/variable"


    # construct header
    headers = CaseInsensitiveDict()
    headers['Authorization'] = f'Bearer {datto_access_token}'
    headers['Content-Type'] = 'application/json'
    headers['accept'] = '*/*'


    # construct req body
    json_data = {
        'name': var_info['varName'],
        'value': var_info['value'],
        'masked': var_info['masked'],
    }

    print(f'Request URL: {request_url}')

    resp = requests.put(request_url, headers=headers, json=json_data)

    print(data)

    print(f"response code: {resp.status_code}")
    print(resp.headers)
    print(resp.content)

### MacOS

In [ ]:
df = pd.read_csv(f'{export_folder}.csv')

# Update Account Variables

In [ ]:
for var_info in account_var_std_list:
    # request content response
    request_url = "https://concord-api.centrastage.net/api/v2/account/variable"


    # construct header
    headers = CaseInsensitiveDict()
    headers['Authorization'] = f'Bearer {datto_access_token}'
    headers['Content-Type'] = 'application/json'
    headers['accept'] = '*/*'


    # construct req body
    json_data = {
        'name': var_info['varName'],
        'value': var_info['value'],
        'masked': var_info['masked'],
    }

    print(f'Request URL: {request_url}')

    resp = requests.put(request_url, headers=headers, json=json_data)

    print(data)

    print(f"response code: {resp.status_code}")
    print(resp.headers)
    print(resp.content)